In [51]:
#############
#############  Team Member :
#############  Name : Md Ziauddin Ridoy
#############  Matriculation Number : 220100676
#############    

In [52]:
import findspark
findspark.init()

import pyspark

In [53]:
from pyspark.sql import SparkSession

spark= SparkSession.builder.appName('Covid-19').getOrCreate()

In [54]:
from pyspark.ml.regression import LinearRegression

In [55]:
dataset=spark.read.csv("worldwide_june_july_csv.csv",inferSchema=True,header=True)

In [56]:
dataset.show()

+--------+---------+---------+------+-------------+
|    Date|Confirmed|Recovered|Deaths|Increase rate|
+--------+---------+---------+------+-------------+
|13-06-20|  7755445|  3706353|430695|  1.759518594|
|14-06-20|  7888575|  3777131|434041|  1.716600401|
|15-06-20|  8010360|  3857338|437545|  1.543814948|
|16-06-20|  8149939|  3955169|444334|  1.742480987|
|17-06-20|  8325831|  4073955|449605|  2.158200202|
|18-06-20|  8464706|  4155099|454614|  1.668001669|
|19-06-20|  8645640|  4250107|460883|  2.137510742|
|20-06-20|  8803904|  4365932|465151|  1.830564308|
|21-06-20|  8934807|  4434628|469208|  1.486874459|
|22-06-20|  9072666|  4526333|472779|  1.542943233|
|23-06-20|  9238013|  4630391|478203|  1.822474232|
|24-06-20|  9405649|  4746118|483377|  1.814632649|
|25-06-20|  9583608|  4838921|489928|  1.892043813|
|26-06-20|  9774609|  4945557|494773|  1.992996792|
|27-06-20|  9952423|  5051864|499291|  1.819141819|
|28-06-20| 10117641|  5140899|502450|  1.660078154|
|29-06-20| 1

In [57]:
dataset.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Increase rate: double (nullable = true)



In [58]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [59]:
featureassembler=VectorAssembler(inputCols=["Confirmed", "Recovered", "Deaths"],outputCol="Independent Features")

In [60]:
output=featureassembler.transform(dataset)

In [61]:
output.show()

+--------+---------+---------+------+-------------+--------------------+
|    Date|Confirmed|Recovered|Deaths|Increase rate|Independent Features|
+--------+---------+---------+------+-------------+--------------------+
|13-06-20|  7755445|  3706353|430695|  1.759518594|[7755445.0,370635...|
|14-06-20|  7888575|  3777131|434041|  1.716600401|[7888575.0,377713...|
|15-06-20|  8010360|  3857338|437545|  1.543814948|[8010360.0,385733...|
|16-06-20|  8149939|  3955169|444334|  1.742480987|[8149939.0,395516...|
|17-06-20|  8325831|  4073955|449605|  2.158200202|[8325831.0,407395...|
|18-06-20|  8464706|  4155099|454614|  1.668001669|[8464706.0,415509...|
|19-06-20|  8645640|  4250107|460883|  2.137510742|[8645640.0,425010...|
|20-06-20|  8803904|  4365932|465151|  1.830564308|[8803904.0,436593...|
|21-06-20|  8934807|  4434628|469208|  1.486874459|[8934807.0,443462...|
|22-06-20|  9072666|  4526333|472779|  1.542943233|[9072666.0,452633...|
|23-06-20|  9238013|  4630391|478203|  1.822474232|

In [62]:
output.select("Independent Features").show(5)

+--------------------+
|Independent Features|
+--------------------+
|[7755445.0,370635...|
|[7888575.0,377713...|
|[8010360.0,385733...|
|[8149939.0,395516...|
|[8325831.0,407395...|
+--------------------+
only showing top 5 rows



In [63]:
output.columns

['Date',
 'Confirmed',
 'Recovered',
 'Deaths',
 'Increase rate',
 'Independent Features']

In [64]:
finalized_data=output.select("Independent Features","Increase rate")

In [65]:
finalized_data.show()

+--------------------+-------------+
|Independent Features|Increase rate|
+--------------------+-------------+
|[7755445.0,370635...|  1.759518594|
|[7888575.0,377713...|  1.716600401|
|[8010360.0,385733...|  1.543814948|
|[8149939.0,395516...|  1.742480987|
|[8325831.0,407395...|  2.158200202|
|[8464706.0,415509...|  1.668001669|
|[8645640.0,425010...|  2.137510742|
|[8803904.0,436593...|  1.830564308|
|[8934807.0,443462...|  1.486874459|
|[9072666.0,452633...|  1.542943233|
|[9238013.0,463039...|  1.822474232|
|[9405649.0,474611...|  1.814632649|
|[9583608.0,483892...|  1.892043813|
|[9774609.0,494555...|  1.992996792|
|[9952423.0,505186...|  1.819141819|
|[1.0117641E7,5140...|  1.660078154|
|[1.027351E7,52358...|   1.54056662|
|[1.0447879E7,5353...|  1.697268022|
|[1.0664168E7,5469...|  2.070171372|
|[1.0871926E7,5754...|  1.948187613|
+--------------------+-------------+
only showing top 20 rows



In [66]:
train_data,test_data=finalized_data.randomSplit([0.70,0.30])

In [67]:
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Increase rate')
regressor=regressor.fit(train_data)

In [68]:
#The Coefficients
regressor.coefficients


DenseVector([-0.0, 0.0, 0.0])

In [69]:
#The intercept
regressor.intercept

-1.494137125244977

In [70]:
pred_results=regressor.evaluate(test_data)

In [71]:
pred_results.predictions.show()

+--------------------+-------------+------------------+
|Independent Features|Increase rate|        prediction|
+--------------------+-------------+------------------+
|[7888575.0,377713...|  1.716600401|1.7088819793793923|
|[8464706.0,415509...|  1.668001669|1.7849395898548146|
|[8645640.0,425010...|  2.137510742|1.8035093662015196|
|[9072666.0,452633...|  1.542943233|1.8087806981863352|
|[9774609.0,494555...|  1.992996792|1.8507897849940431|
|[9952423.0,505186...|  1.819141819|1.8455109907481442|
|[1.027351E7,52358...|   1.54056662|1.8143927625913892|
|[1.0447879E7,5353...|  1.697268022|1.8220719290188205|
|[1.0871926E7,5754...|  1.948187613|1.8141786960927704|
|[1.1076086E7,5863...|  1.877864143|1.8049162597245696|
|[1.1620096E7,6302...|  1.466094363|1.7673083921273494|
|[1.310329E7,72573...|  1.494404841| 1.689342513222485|
+--------------------+-------------+------------------+



In [72]:
#Calculate RMSE (Root Mean Square Error)
regressor.summary.rootMeanSquaredError

0.1600222492722284

In [73]:
#Calculate r2 (R squared)
regressor.summary.r2

0.0663173075302339

In [74]:
from pyspark.ml.evaluation import RegressionEvaluator
RegessorEval = RegressionEvaluator(labelCol="Increase rate", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = RegessorEval.evaluate(pred_results.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = RegessorEval.evaluate(pred_results.predictions, {RegessorEval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = RegessorEval.evaluate(pred_results.predictions, {RegessorEval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = RegessorEval.evaluate(pred_results.predictions, {RegessorEval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 0.196
MSE: 0.038
MAE: 0.166
r2: 0.103


In [49]:
spark.stop()